In [33]:
import os

import numpy as np

In [34]:
def Input(day):
    # Read an input
    filename = 'data/{}.txt'.format(day)
    return open(filename)

def Lines(day):
    # Read the input and convert to an array of strings.
    return Input(day).read().splitlines()

def vector(day):
    # Read the input, and convert it to a vector of numbers
    return list(map(float, Lines(day)))

## Day 1

In [35]:
# Read input
inp = vector(1)
           
# The answer to the first one is just the sum
print(sum(inp))

533.0


In [36]:
# For the second part, use a set to determine if we've seen 
# a frequency before
v = set()
ii = 0
s = 0
while not found:
    s = s + inp[ii % len(inp)]
    if s in v:
        print("Found freq: {} at iteration {} ({} passes)".format(
                s, ii, ii % len(inp)))
        break
    v.add(s)
    ii = ii + 1

Found freq: 73272.0 at iteration 130408 (528 passes)
